In [ ]:
with open('train_embedding.txt', 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    # 假设每行数据由空格分隔
    values = line.strip().split()
    # 将字符串转换为浮点数
    values = [float(val) for val in values]
    data.append(values)

# 将数据转换为张量
train_embedding = data
file.close()

with open('train_result.txt', 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    # 假设每行数据由空格分隔
    values = line.strip().split()
    # 将字符串转换为浮点数
    values = [float(val) for val in values]
    data.append(values)

# 将数据转换为张量
train_result = data
file.close()

with open('test_embedding.txt', 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    # 假设每行数据由空格分隔
    values = line.strip().split()
    # 将字符串转换为浮点数
    values = [float(val) for val in values]
    data.append(values)

# 将数据转换为张量
test_embedding = data
file.close()

with open('test_result.txt', 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    # 假设每行数据由空格分隔
    values = line.strip().split()
    # 将字符串转换为浮点数
    values = [float(val) for val in values]
    data.append(values)

# 将数据转换为张量
test_result = data
file.close()

In [ ]:
X_train, Y_train = np.asarray(train_embedding), np.asarray(train_result)
X_test, Y_test = np.asarray(test_embedding), np.asarray(test_result)

In [ ]:
import os
import math

from matplotlib import pyplot as plt
import torch
import gpytorch
from gpytorch import kernels, means, models, mlls, settings
from gpytorch import distributions as distr
import torch.optim as optim
import numpy as np

train_embedding = torch.tensor(np.load('train_embedding.npy'))
train_result = torch.tensor(np.log(np.load('train_result.npy')))
test_embedding = torch.tensor(np.load('test_embedding.npy'))
test_result = torch.tensor(np.log(np.load('test_result.npy')))

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = means.ConstantMean()
        self.covar_module = kernels.ScaleKernel(kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return distr.MultivariateNormal(mean_x, covar_x)


# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_embedding, train_result, likelihood)

In [ ]:
norm_max_label = 22.956885487470185
norm_min_label = 0.0
def unnormalize_torch(vals):
    vals = (vals * (norm_max_label - norm_min_label)) + norm_min_label
    return torch.exp(vals)

def loss_function(predict, label):
    qerror = []
    predict = unnormalize_torch(predict)

    for i in range(len(label)):
        if (predict[i] > label[i]):
            qerror.append(predict[i] / label[i])
        else:
            qerror.append(label[i] / predict[i])

    return torch.mean(torch.cat(qerror))


In [ ]:
training_iter = 50

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
# mll = mlls.ExactMarginalLogLikelihood(likelihood, model)
mll = mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_embedding)
    # Calc loss and backprop gradients
    loss = -mll(output, train_result)
    print(loss.shape)
    loss_value = loss.item()  # 获取损失值

    loss_value.backward()  # 计算梯度
    # loss.backward()
    if i % 5 == 4:
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item()
        ))
    optimizer.step()

In [ ]:
from argparse import ArgumentParser, FileType, ArgumentDefaultsHelpFormatter
import os
import datetime
import sys
from absl import app
from absl import flags
from jax import grad
from functools import partial
import jax.numpy as np
import numpy as onp
# from jax.api import jit
import jax.random as rand
from jax import vmap
from jax.lib import xla_bridge
# from jax.config import config
import jax.scipy as scipy
from neural_tangents import stax
import neural_tangents as nt
# import datasets
# import schemas
from util import draw_uncertainty, calibration_plot, PredictionStatistics, draw_kernel_heatmap, show_memory_usage
from util import uneven_train_test_split, train_test_val_split
import torch

pred_stat = PredictionStatistics()

norm_max_label = 22.956885487470185
norm_min_label = 0.0

train_embedding = np.load('train_embedding.npy')
train_result = np.log(np.load('train_result.npy'))/(norm_max_label-norm_min_label)
train_result_real = np.load('train_result.npy')
test_embedding = np.load('test_embedding.npy')
test_result = np.log(np.load('test_result.npy'))/(norm_max_label-norm_min_label)
test_result_real = np.load('test_result.npy')

In [ ]:
# train_embedding_last_1000 = train_embedding[-396:]
# test_embedding = np.concatenate([test_embedding, train_embedding_last_1000], axis=0)
# train_embedding = train_embedding[:-396]

# train_result_last_1000 = train_result[-396:]
# test_result = np.concatenate([test_result, train_result_last_1000], axis=0)
# train_result = train_result[:-396]

# train_result_real_last_1000 = train_result_real[-396:]
# test_result_real = np.concatenate([test_result_real, train_result_real_last_1000], axis=0)
# train_result_real = train_result_real[:-396]

In [ ]:
print(train_embedding.shape)
print(train_result.shape)
print(train_result_real.shape)
print(test_embedding.shape)
print(test_result.shape)
print(test_result_real.shape)

In [ ]:
def unnormalize_torch(vals):
    vals = (vals * (norm_max_label - norm_min_label)) + norm_min_label
    return np.exp(vals)

def loss_function(predict, label):
    qerror = []
    predict = unnormalize_torch(predict)

    for i in range(len(label)):
        # print(predict[i], label[i])

        if (predict[i] > label[i]):
            qerror.append(predict[i] / label[i])
        else:
            qerror.append(label[i] / predict[i])
    
    return np.mean(np.concatenate(qerror))

In [ ]:
def NNGP_train_and_test(X_train, Y_train, Y_train_real, X_test, Y_test, Y_test_real, query_infos_train = None, query_infos_test= None):

	def prediction(pred_fn, X_test, kernel_type="nngp", compute_cov = True):
		pred_mean, pred_cov = pred_fn(x_test=X_test, get=kernel_type, compute_cov= compute_cov)
		return pred_mean, pred_cov

	init_fn, apply_fn, kernel_fn = stax.serial(
		stax.Dense(1024), stax.Relu(), 
		stax.Dense(1)
	)
	kernel_fn = nt.batch(kernel_fn, device_count = 0, batch_size = 0)
	# predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, X_test, Y_test, diag_reg = 1e-2)
	predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, np.concatenate([X_train, X_test], axis=0), np.concatenate([Y_train, Y_test], axis=0), diag_reg = 1e-1/7)
	

	# 验证训练集训练结果
	pred_mean_train, pred_cov_train = prediction(predict_fn, X_train, kernel_type = 'nngp')
	pred_std_train = np.sqrt(np.diag(pred_cov_train))
	q_error_train = loss_function(pred_mean_train, Y_train_real)
	print("q_error_train: {}".format(q_error_train))

	# 验证测试集
	pred_mean_test, pred_cov_test = prediction(predict_fn, X_test, kernel_type = 'nngp')
	pred_std_test = np.sqrt(np.diag(pred_cov_test))
	q_error_test = loss_function(pred_mean_test, Y_test_real)
	print("q_error_test: {}".format(q_error_test))
	
	# mse = np.sum(np.power(pred_mean -Y_test, 2))
	# #print(mse)
	# print("Mean Square Error: {}".format(mse))

	#Obtain the inference time
	# print(X_test.shape, Y_test.shape)
	# start = datetime.datetime.now()
	# pred_mean, pred_cov = prediction(predict_fn, X_test, kernel_type = 'nngp')
	# end = datetime.datetime.now()
	# duration = (end - start).total_seconds()
	# print("Inference time={} seconds".format(duration))


	# errors = onp.ravel(onp.array(pred_mean - Y_test))
	# pred_std = onp.ravel(onp.array(pred_std))
	# outputs = onp.ravel(onp.array(pred_mean))
	# Y_test = onp.ravel(onp.array(Y_test))

In [ ]:
NNGP_train_and_test(train_embedding, train_result, train_result_real, test_embedding, test_result, test_result_real, None, None)